### 1. Import Necessary Libraries

In [119]:
import sys
import nltk
import sklearn
import pandas
import numpy



In [120]:
print('Python: {}'.format(sys.version))
print('NLTK: {}'.format(nltk.__version__))
print('Scikit-learn: {}'.format(sklearn.__version__))
print('pandas: {}'.format(pandas.__version__))
print('numpy: {}'.format(numpy.__version__))

Python: 3.7.10 (default, Feb 26 2021, 10:16:00) 
[Clang 10.0.0 ]
NLTK: 3.5
Scikit-learn: 0.24.2
pandas: 1.3.3
numpy: 1.21.6


### 2. Load the Dataset

In [121]:
import pandas as pd
import numpy as np

#load the dataset of SMS message
df = pd.read_table('SMSSpamCollection',header=None,encoding='utf-8')

In [122]:
#print useful information about the dataset
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       5572 non-null   object
 1   1       5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [123]:
#check the class distribution
classes = df[0]
print(classes.value_counts())

ham     4825
spam     747
Name: 0, dtype: int64


### 2. Preprocess the Data

Preprocessing the data is an essential step in natural language process. In the following cells, we will convert our class labels to binary values using the LabelEncoder from sklearn, replace email addresses, URLs, phone numbers, and other symbols by using regular expressions, remove stop words, and extract word stems.

In [124]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
Y = encoder.fit_transform(classes)
print(Y[:10])

[0 0 1 0 0 1 0 0 1 1]


In [125]:
#store the SMS message Data
text_message = df[1]
print(text_message[:10])

0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
5    FreeMsg Hey there darling it's been 3 week's n...
6    Even my brother is not like to speak with me. ...
7    As per your request 'Melle Melle (Oru Minnamin...
8    WINNER!! As a valued network customer you have...
9    Had your mobile 11 months or more? U R entitle...
Name: 1, dtype: object


### 2.1 Regular Expressions


use regular expressions to replace email addresses, URLs, phone numbers, other numbers

In [126]:
#Replace email addresses with 'email'
processed = text_message.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$','emailaddress')

#Replace URLs with webaddress
precessed = processed.str.replace(r'http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$','webaddress')

# Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
processed = processed.str.replace(r'£|\$', 'moneysymb')
    
# Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  'phonenumbr')
    
# Replace numbers with 'numbr'
processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

# Remove punctuation
processed = processed.str.replace(r'[^\w\d\s]', ' ')

# Replace whitespace between terms with a single space
processed = processed.str.replace(r'\s+', ' ')

# Remove leading and trailing whitespace
processed = processed.str.replace(r'^\s+|\s+?$', '')

/Users/ravibabariya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/ravibabariya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/ravibabariya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  
/Users/ravibabariya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/Users/ravibabariya/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app
/Users/ravibabar

In [127]:
# change words to lower case - Hello, HELLO, hello are all the same word
processed = processed.str.lower()
print(processed)

0       go until jurong point crazy available only in ...
1                                 ok lar joking wif u oni
2       free entry in numbr a wkly comp to win fa cup ...
3             u dun say so early hor u c already then say
4       nah i don t think he goes to usf he lives arou...
                              ...                        
5567    this is the numbrnd time we have tried numbr c...
5568                  will ü b going to esplanade fr home
5569    pity was in mood for that so any other suggest...
5570    the guy did some bitching but i acted like i d...
5571                            rofl its true to its name
Name: 1, Length: 5572, dtype: object


In [128]:
#nltk.download()

In [137]:
from nltk.corpus import stopwords

#remove stop word from text messages
stop_words = set(stopwords.words('english'))

processed = processed.apply(lambda x:' '.join(term for term in x.split() if term not in stop_words))


In [138]:
#remove word stems using a porter stemmer
ps = nltk.PorterStemmer()

processed = processed.apply(lambda x:' '.join(ps.stem(term) for term in x.split()))
print(processed)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri numbr wkli comp win fa cup final tk...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    numbrnd time tri numbr contact u u moneysymbnu...
5568                              ü b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth el next...
5571                                       rofl true name
Name: 1, Length: 5572, dtype: object


### 3. Generating Features

Feature engineering is the process of using domain knowledge of the data to create features for machine learning algorithms. In this project, the words in each text message will be our features. For this purpose, it will be necessary to tokenize each word. We will use the 1500 most common words as features.

In [131]:
from nltk.tokenize import word_tokenize

#creating bag-of-word
all_words = []

for message in processed:
    words = word_tokenize(message)
    for w in words:
        all_words.append(w)
        
all_words = nltk.FreqDist(all_words)


In [165]:
# print the total number of words and the 15 most common words
print('Number of words: {}'.format(len(all_words)))
print('Most common word: {}'.format(all_words.most_common(1500)))

Number of words: 6579
Most common word: [('numbr', 2648), ('u', 1207), ('call', 674), ('go', 456), ('get', 451), ('ur', 391), ('gt', 318), ('lt', 316), ('come', 304), ('moneysymbnumbr', 303), ('ok', 293), ('free', 284), ('day', 276), ('know', 275), ('love', 266), ('like', 261), ('got', 252), ('time', 252), ('good', 248), ('want', 247), ('text', 231), ('send', 214), ('txt', 190), ('need', 190), ('one', 185), ('today', 181), ('take', 174), ('ü', 173), ('see', 173), ('stop', 168), ('home', 167), ('think', 166), ('repli', 163), ('r', 162), ('lor', 162), ('sorri', 160), ('still', 158), ('tell', 157), ('n', 155), ('numbrp', 154), ('back', 153), ('mobil', 153), ('da', 151), ('dont', 149), ('make', 148), ('k', 147), ('week', 141), ('pleas', 141), ('phone', 141), ('say', 140), ('hi', 140), ('work', 136), ('new', 136), ('pl', 135), ('later', 135), ('hope', 134), ('miss', 133), ('ask', 133), ('co', 131), ('meet', 128), ('msg', 127), ('messag', 125), ('night', 124), ('dear', 122), ('c', 121), ('wa

In [141]:
# use the 1500 most common words as features
word_features = list(all_words.keys())[:1500]


In [152]:
# The find_features function will determine which of the 1500 word features are contained in the review
def find_features(message):
    words = word_tokenize(message)
    features = {}
    for word in word_features:
        features[word] = (word in words)

    return features



In [176]:
# Lets see an example!
features = find_features(processed[5])
for key,value in features.items():
    if value == True:
        print(key)

ok
numbr
std
freemsg
hey
darl
week
word
back
like
fun
still
tb
xxx
chg
send
moneysymbnumbr
rcv


In [193]:
#now lets do it for all messages
message = list(zip(processed, Y))

# define a seed for reproducibility
seed = 1
np.random.seed = seed
np.random.shuffle = message

# call find_features function for each SMS message
featureset = [(find_features(text), label) for (text, label) in message ]

In [197]:
len(featureset)

5572

In [198]:
# we can split the featuresets into training and testing datasets using sklearn
from sklearn import model_selection

# split the data into training and testing datasets
training, testing = model_selection.train_test_split(featureset, test_size=0.25, random_state=seed)

In [201]:
print(len(training))
print(len(testing))

4179
1393


### 4. Scikit-Learn Classifiers with NLTK

Now that we have our dataset, we can start building algorithms! Let's start with a simple linear support vector classifier, then expand to other algorithms. We'll need to import each algorithm we plan on using from sklearn. We also need to import some performance metrics, such as accuracy_score and classification_report.

In [202]:
# We can use sklearn algorithms in NLTK
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import SVC


model = SklearnClassifier(SVC(kernel='linear'))

#train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print('SVC accuracy: {}'.format(accuracy) )

SVC accuracy: 98.56424982053123


In [206]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Define models to train
names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=40),
    LogisticRegression(),
    SGDClassifier(max_iter=100),
    MultinomialNB(),
    SVC(kernel='linear')
]

models = list(zip(names, classifiers))

for name, model in models:
    nltk_model = SklearnClassifier(model)
    nltk_model.train(training)
    accuracy = nltk.classify.accuracy(nltk_model,testing)*100
    print('{} Accuracy: {}'.format(name, accuracy))

K Nearest Neighbors Accuracy: 94.75951184493898
Decision Tree Accuracy: 97.34386216798278
Random Forest Accuracy: 98.99497487437185
Logistic Regression Accuracy: 98.63603732950466
SGD Classifier Accuracy: 98.63603732950466
Naive Bayes Accuracy: 98.77961234745155
SVM Linear Accuracy: 98.56424982053123


In [216]:
# Ensemble methods - Voting classifier
from sklearn.ensemble import VotingClassifier

names = ["K Nearest Neighbors", "Decision Tree", "Random Forest", "Logistic Regression", "SGD Classifier",
         "Naive Bayes", "SVM Linear"]

classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=54),
    LogisticRegression(),
    SGDClassifier(max_iter = 100),
    MultinomialNB(),
    SVC(kernel = 'linear')
]

models = list(zip(names, classifiers))

nltk_ensamble = SklearnClassifier(VotingClassifier(estimators=models, voting='hard', n_jobs=-1))
nltk_ensamble.train(training)
accuracy= nltk.classify.accuracy(nltk_ensamble,testing)*100
print('Votion Classifier: Accuracy: {}'.format(accuracy))


Votion Classifier: Accuracy: 98.85139985642498


In [218]:
# make class label prediction for testing set
txt_features, label = zip(*testing)

prediction= nltk_ensamble.classify_many(txt_features)

In [223]:
# print a confusion matrix and a classification report
print(classification_report(label, prediction))

pd.DataFrame(confusion_matrix(label,prediction),index= [['actual', 'actual'],['ham','spam']],columns = [['predicted','predicted'],['ham','spam']])

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1208
           1       0.98      0.94      0.96       185

    accuracy                           0.99      1393
   macro avg       0.98      0.97      0.97      1393
weighted avg       0.99      0.99      0.99      1393



predicted     
                  ham spam
actual ham       1204    4
       spam        12  173